In [1]:
!pip install transformers
!pip install torch
!pip install pickle5


In [2]:
!pip install torch

In [3]:
!pip install pickle5


^C


In [4]:
!pip install datasets

In [5]:
!pip install jiwer

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import pandas as pd 
import pickle

global_path =  'D:/Courses spring/ML System Design/movie_posters_captioning/'
karpathy_json_path = global_path + 'data/interim/films_df.pkl'
empty_imges = global_path + 'notebooks/evgeennnn/' +  'empty_img.txt'

with open(karpathy_json_path, "rb") as dt:
      data = pickle.load(dt) 

      x = data['description'].apply(lambda x: len(x))
      data = data[x.between(x.quantile(.1), x.quantile(.9))]
#       f = open(empty_imges, "r")
#       d = f.read()
#       empty_img = list(map(int, d.split(', ')))
#       print("empty_img", len(empty_img))
#       print("data", data.shape)
      
#       mask = data['filmId'].isin(empty_img)
#       data = data[~mask]

      print("data", data.shape)

data (23583, 11)


In [8]:
data.reset_index(inplace=True)
data = data[:500]

In [9]:
# data.to_csv(global_path + 'data/interim/films_df.csv') 

In [2]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.2)


In [3]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
# import torch
# from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, BertTokenizer

# # model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
#     "google/vit-base-patch16-224-in21k", "DeepPavlov/rubert-base-cased"
# )
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
# tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# print('')

In [4]:
global_path =  'D:/Courses spring/ML System Design/movie_posters_captioning/'
image_path = global_path + '/data/img/2.jpg'


# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id


In [5]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df,feature_extractor,tokenizer, max_target_length=512):
        self.root_dir = root_dir
        self.df = df
        # self.processor = processor #feature_extractor
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = str(self.df['filmId'][idx]) + '.jpg'
        text = self.df['description'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        image = image.resize([384, 384])
        pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
       
        labels = self.tokenizer(text, 
                                          truncation='longest_first',
                                          padding="max_length", 
                                          # return_tensors="pt",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [6]:
from transformers import TrOCRProcessor, ViTFeatureExtractor, BertTokenizer, VisionEncoderDecoderModel
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "DeepPavlov/rubert-base-cased"
)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

device = 'cuda:0'
model.to(device)

train_dataset = IAMDataset(root_dir=global_path + 'data/img/',
                           df=train_df,
                          #  processor=processor
                           feature_extractor=feature_extractor,
                           tokenizer=tokenizer
                           )
eval_dataset = IAMDataset(root_dir=global_path + 'data/img/',
                           df=test_df,
                          #  processor=processor
                           feature_extractor=feature_extractor,
                           tokenizer=tokenizer
                          )

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['bert.encoder.layer.7.crossattention.output.dense.bias', 'bert.encoder.layer.8.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.9.crossattention.self.query.weight', 'bert.encoder.layer.3.crossattention.self.

In [7]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 18866
Number of validation examples: 4717


In [26]:
encoding = train_dataset[0]
for k,v in encoding.items():
    print(k, v.shape)

pixel_values torch.Size([3, 224, 224])
labels torch.Size([512])


In [8]:
image = Image.open(train_dataset.root_dir + str(train_df['filmId'][0]) + '.jpg').convert("RGB")
# image

In [9]:
from transformers import VisionEncoderDecoderModel

# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    # num_train_epochs=3,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=50,
    eval_steps=20,
)

In [10]:
from datasets import load_metric

cer_metric = load_metric("cer")

C:\Users\USSRER\AppData\Local\Temp\ipykernel_5692\152175726.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [11]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [43]:
import torch
torch.cuda.empty_cache()

In [12]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor   
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

Using cuda_amp half precision backend
C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18866
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 56598


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\USSRER\AppData\Local\Temp\ipykernel_5692\1026679545.py", line 13, in <module>
    trainer.train()
  File "C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 1500, in train
    return inner_training_loop(
  File "C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 1742, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 2496, in training_step
    self.scaler.scale(loss).backward()
  File "C:\Users\USSRER\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_tensor.py", line 488, in backward
  

In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

images = []
image_path = global_path + 'data/img/1.jpg';
i_image = Image.open(image_path)
i_image = i_image.resize([384,384])
if i_image.mode != "RGB":
  i_image = i_image.convert(mode="RGB")

# model.feat
images.append(i_image)
pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

output_ids = model.generate(pixel_values, **gen_kwargs)
print(output_ids)
print(tokenizer.vocab)
preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
preds = [pred.strip() for pred in preds]
preds

In [ ]:
i_image

In [ ]:
trainer.save_model(global_path + 'model_ru')